# England Internal Report
This notebook generates the monthly performance report using the **England_Internal** configuration profile.

In [ ]:
import sys
import os
import pandas as pd

# 1. Setup Path to Source
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..', 'src')))

# 2. Import Libraries
from enterprise_writer import EnterpriseExcelWriter
from query_library import fGetRegionalSales, fGetDataDictionary
from config_provider import fGetReportConfig

### 1. Load Configuration
We load the `England_Internal` profile. This pulls the Red color scheme (`#CE1124`) and the official prefixes.

In [ ]:
# Define the profile we want
vProfileName = 'England_Internal'

# Fetch config from DB
vConfig = fGetReportConfig(vProfileName)

print(f"Loaded Profile: {vProfileName}")
print(f"Theme Color: {vConfig.get('primary_color')}")

### 2. Generate Report
We initialize the writer with the config. Note how we use `vConfig.get('key')` to dynamically apply settings.

In [ ]:
# Get Data
dfSales = fGetRegionalSales("North") # Filter for England/North data if needed
dfDict = fGetDataDictionary()

# Initialize Writer
vReport = EnterpriseExcelWriter("England_Performance_Report.xlsx", vConfig=vConfig)
vReport.fSetColumnMapping(dfDict)

# --- Apply Configured Visuals ---

# 1. Add Logo (If defined in config)
if vConfig.get('logo_path'):
    vReport.fAddLogo(vConfig['logo_path'], vWidthScale=0.4)

# 2. Add Title (With Configured Prefix)
vTitle = f"{vConfig.get('title_prefix', '')}Monthly Summary"
vReport.fAddTitle(vTitle)

# 3. Add Watermark (If flag is True)
if vConfig.get('show_watermark') == 'True':
    # Assumes you have a file named 'draft_watermark.png' in assets
    vReport.fAddWatermark("assets/draft_watermark.png")

# Write Data
vReport.fWriteDataframe(dfSales, vAddTotals=True)

# Apply Conditional Formatting using the Secondary Color from Config
vSecondaryColor = vConfig.get('secondary_color', '#FF9999') # Default to pink if missing
vReport.fAddConditionalFormat('efficiency_rate', 'cell', {'criteria': '<', 'value': 0.90}, vColor=vSecondaryColor)

# Finish
vReport.fClose()